In [ ]:
import os
if os.getcwd().endswith('lab02_linear'):
    os.chdir('..')

In [ ]:
import sys
import math

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from lab02_linear.common.data import read
from lab02_linear.common.graders import prediction, smape

fin = open('lab02_linear/resources/LR/5.txt')
m = int(fin.readline().strip())
n1, xtrain, ytrain = read(fin)
n2, xtest, ytest = read(fin)

## Сразу хорошие результаты

Покажем, что градиентный спуск вполне себе неплох и умеет добиваться неплохих результатов при определенных параметрах

In [ ]:
%%time

from lab02_linear.methods.gradient import solve as g_solve

alpha, score = next(g_solve(
    10000, xtrain, ytrain,
    do_normalize=False, fill='zero', lmbd=lambda step: 0.8 / math.pow(step + 1, 0.3)
))
yptest = prediction(alpha, xtest)
print(score, '-', smape(yptest, ytest))

## Теперь чуть более упорядоченно

### 1. Least squares

In [ ]:
from lab02_linear.methods.least_squares import solve as ls_solve

for do_normalize in [True, False]:
    try:
        alpha, score, err = ls_solve(xtrain, ytrain, do_normalize)
        yptest = prediction(alpha, xtest)
        print(score, '-', smape(yptest, ytest), '/', err)
    except np.linalg.LinAlgError:
        print('fail')

Добавим небольшую ridge-составляющую для случая плохой обсуловленности:

In [ ]:
for do_normalize in [True, False]:
    for ridge in np.hstack(([0], np.power(10., range(-20, 1, 3)))):
        try:
            alpha, score, err = ls_solve(xtrain, ytrain, do_normalize, ridge)
            yptest = prediction(alpha, xtest)
            print(f'norm: {do_normalize}, ridge: {ridge} ->', '%.9f' % score, '-', '%.9f' % smape(yptest, ytest), '/', err)
        except np.linalg.LinAlgError:
            print(f'norm: {do_normalize}, ridge: {ridge} -> fail')

Как видно, у такого подхода достаточно большие проблемы, поэтому стоит попробовать сделать регрессию с помощью SVD-декомпозиции. Переберем все возможные параметры, которые в целом можно настраивать (нормализацию и ридж-составляющую):

In [ ]:
from lab02_linear.methods.least_squares import solve_svd as svd_solve

for do_normalize in [True, False]:
    for ridge in np.hstack(([0], np.power(10., range(-8, 1, 2)))):
        try:
            alpha, score, err = svd_solve(xtrain, ytrain, do_normalize, ridge)
            yptest = prediction(alpha, xtest)
            print(f'norm: {do_normalize}, ridge: {ridge} ->', '%.9f' % score, '-', '%.9f' % smape(yptest, ytest), '/', err)
        except np.linalg.LinAlgError:
            print(f'norm: {do_normalize}, ridge: {ridge} -> fail')

Заметим, что поведение примерно ожидаемое - при SVD-декомпозиции в ридж-составляющей нет нужды, более того, нет ошибок о невозможности инвертировать матрицу.

In [ ]:
%%time
_, _, _ = svd_solve(xtrain, ytrain, True, 0)

### 2. Градиентный спуск

Здесь можно перебирать коэффициент при градиенте и его затухание, изначальное заполнение, нормализацию и регуляризацию. Поскольку полный перебор будет работать слишком долго, понадеемся на рандом и сделаем ограниченное число шагов. 1000 итераций работает примерно секунду - попробуем сначала запустить с таким ограничением, подберем оптимальные параметры и дальше будем перебирать число итераций.

In [ ]:
steps = 20

total_norm = [0, 0]
total_fill = [0, 0, 0]
total_reg = [0, 0, 0]

for t1, do_normalize in enumerate([True, False]):
    for t2, fill in enumerate(['zero', 'uniform', 'smart']):
        for t3, reg in enumerate([0, 0.0001, 0.01]):
            for i in range(steps):
                c = np.random.uniform(0.1, 1)
                deg = np.random.uniform(-4, 4)
                p = 2 * np.random.uniform(0, 1) ** 2
                alpha, score = next(g_solve(
                    1000, xtrain, ytrain,
                    do_normalize, fill, lmbd=lambda step: c * (math.pow(10., deg)) / math.pow(step + 1, p)
                ))
                yptest = prediction(alpha, xtest)
                score = smape(yptest, ytest)
                if score < 0.1:
                    print(f'{score}: {do_normalize}, {fill}, {reg}, {c}x10^{deg}/t^{p}')
                    
                total_norm[t1] += score
                total_fill[t2] += score
                total_reg[t3] += score
            print('+1/18')

In [ ]:
print(total_norm, total_fill, total_reg)

Найти что-то хорошее не получилось, но можно попробовать зафиксировать какие-то параметры и подвигать другие. Более того, будем запоминать лучшее из встреченных решений по пути (сначала проверим на известном хорошем решении):

In [ ]:
%%time

from lab02_linear.methods.gradient import memoized_solve as mg_solve

alpha, score = mg_solve(
    10000, xtrain, ytrain,
    do_normalize=False, fill='zero', lmbd=lambda step: 0.8 / math.pow(step + 1, 0.3)
)
yptest = prediction(alpha, xtest)
print(score, '-', smape(yptest, ytest))

Если честно, то и руками, и какими-то минимальными переборами получить что-то лучше не очень выходит, поэтому можно остановиться на этом варианте и построить для него графики:

In [ ]:
train_scores, test_scores = [], []
for alpha, score in g_solve(10000, xtrain, ytrain, sequential=True):
    yptest = prediction(alpha, xtest)
    train_scores.append(score)
    test_scores.append(smape(yptest, ytest))

In [ ]:
plt.plot(train_scores)
plt.plot(test_scores, color='red')

Как видно, градиентный спуск действительно сходится в какой-то локальный минимум, но, судя по скору, это не глобальный минимум. Можно было бы как-то подобрать оптимальные параметры, но пока что ни один адекватный поиск до них не дошел.

### 3. Отжиг

In [ ]:
from lab02_linear.methods.annealing import solve

alpha, score = next(solve(100000, xtrain, ytrain))
print(score)

In [ ]:
train_scores_a, test_scores_a = [], []
for alpha, score in solve(10000, xtrain, ytrain, sequential=True):
    yptest = prediction(alpha, xtest)
    train_scores_a.append(score)
    test_scores_a.append(smape(yptest, ytest))

In [ ]:
plt.plot(train_scores)
plt.plot(test_scores, color='red')
plt.plot(train_scores_a, color='green')
plt.plot(test_scores_a, color='yellow')

Параметры отжига можно настраивать еще дольше, чем параметры градиентного спуска (force_downhill, temperature, mutations), поэтому можно ограничиться текущим графиком, сказав, что он застрял в менее удачном локальном минимуме.

## Вывод

Наиболее точный метод - метод наименьших квадратов. Его использование сопряжено с тем, что для большинства матриц вычисленное напрямую значение псевдообратной будет некорректно из-за потерь в точности и несогласованности данных. Для этого + для ускорения программы можно использовать разложения (QR, SVD). На достаточно неплохих данных такой метод сразу дает наилучший результат без необходимости проводить какие-либо итерации.

Метод градиентного спуска гарантированно (при правильной настройке) сходится в некоторый локальный минимум, однако чтобы он сошелся в глобальный, требуется очень тщательно подбирать параметры и выбирать начальные приближения, что не всегда возможно в реалиях ограниченного времени.

Итерационные генетические алгоритмы или тот же отжиг напоминают градиентный спуск тем, что постепенно приближаются к оптимальному значению, однако так же требуют гибкой настройки.
Преимущество градиентного спуска в том, что по графику его результатов проще понять, как надо перенастроить параметры, чтобы достичь лучшего результата, тогда как зависимость между параметрами генетического алгоритма и его поведением обычно менее очевидна и связь с задачей менее интерпретируема.
В случае отжига можно провести параллель между температурой и затухающим learning_rate.